In [4]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import scipy.io as sio
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings('ignore')

# contruct data

In [5]:
def read_subject_list(txt_path):
    with open(txt_path, "r") as f:
        subs = [line.strip() for line in f if line.strip()]
    return subs


def build_df_atlas(KL_mat, human_ids, nonhuman_ids, species, atlas, hemisphere, region):
    """
    KL_mat: numpy array (n_human × n_nonhuman)
    human_ids: list of str (len=n_human)
    nonhuman_ids: list of str (len=n_nonhuman)
    species: str, e.g. "chimpanzee"
    hemisphere: str, "LH" or "RH"
    region: str
    """
    n_h, n_x = KL_mat.shape
    if len(human_ids) != n_h or len(nonhuman_ids) != n_x:
        raise ValueError(f"Shape mismatch for {species}-{hemisphere}: KL {KL_mat.shape}, human {len(human_ids)}, nonhuman {len(nonhuman_ids)}")

    rows = []
    for i, h in enumerate(human_ids):
        for j, x in enumerate(nonhuman_ids):
            rows.append({
                "human_id": h,
                "nonhuman_id": x,
                "species": species,
                "atlas": atlas,
                "hemisphere": hemisphere,
                "region": region,
                "KL": float(KL_mat[i, j])
            })
    return pd.DataFrame(rows)

In [6]:
human_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/humanlist40.txt"
chimp_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/chimplist46.txt"
macaque_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/macaque_tvb_list.txt"
marmoset_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/marmoset_MBM_list.txt"

human_ids     = read_subject_list(human_list_path)     # 40
chimp_ids     = read_subject_list(chimp_list_path)     # 46
macaque_ids   = read_subject_list(macaque_list_path)   # 8
marmoset_ids  = read_subject_list(marmoset_list_path)  # 24

In [ ]:
i = 17
region = "A45c"

roi_index_human = i - 1

datapath = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/minKL_indi'

KL_human_chimp_LH = sio.loadmat(f'{datapath}/c2h_minKL_on_human_atlas_all_subject.L.mat')
KL_human_chimp_LH = KL_human_chimp_LH["minKL_c2h_all_L"]
KL_human_chimp_LH = KL_human_chimp_LH[:,:,roi_index_human]
KL_human_chimp_RH = sio.loadmat(f'{datapath}/c2h_minKL_on_human_atlas_all_subject.R.mat')
KL_human_chimp_RH = KL_human_chimp_RH["minKL_c2h_all_R"]
KL_human_chimp_RH = KL_human_chimp_RH[:,:,roi_index_human]

KL_human_macaque_LH = sio.loadmat(f'{datapath}/mac2h_minKL_on_human_atlas_all_subject.L.mat')
KL_human_macaque_LH = KL_human_macaque_LH["minKL_mac2h_all_L"]
KL_human_macaque_LH = KL_human_macaque_LH[:,:,roi_index_human]
KL_human_macaque_RH = sio.loadmat(f'{datapath}/mac2h_minKL_on_human_atlas_all_subject.R.mat')
KL_human_macaque_RH = KL_human_macaque_RH["minKL_mac2h_all_R"]
KL_human_macaque_RH = KL_human_macaque_RH[:,:,roi_index_human]

KL_human_marmoset_LH = sio.loadmat(f'{datapath}/mar2h_minKL_on_human_atlas_all_subject.L.mat')
KL_human_marmoset_LH = KL_human_marmoset_LH["minKL_mar2h_all_L"]
KL_human_marmoset_LH = KL_human_marmoset_LH[:,:,roi_index_human]
KL_human_marmoset_RH = sio.loadmat(f'{datapath}/mar2h_minKL_on_human_atlas_all_subject.R.mat')
KL_human_marmoset_RH = KL_human_marmoset_RH["minKL_mar2h_all_R"]
KL_human_marmoset_RH = KL_human_marmoset_RH[:,:,roi_index_human]

KL_human_marmoset_vPaxinos_LH = sio.loadmat(f'{datapath}/mar2h_vPaxinos_minKL_on_human_atlas_all_subject.L.mat')
KL_human_marmoset_vPaxinos_LH = KL_human_marmoset_vPaxinos_LH["minKL_mar2h_all_L"]
KL_human_marmoset_vPaxinos_LH = KL_human_marmoset_vPaxinos_LH[:,:,roi_index_human]
KL_human_marmoset_vPaxinos_RH = sio.loadmat(f'{datapath}/mar2h_vPaxinos_minKL_on_human_atlas_all_subject.R.mat')
KL_human_marmoset_vPaxinos_RH = KL_human_marmoset_vPaxinos_RH["minKL_mar2h_all_R"]
KL_human_marmoset_vPaxinos_RH = KL_human_marmoset_vPaxinos_RH[:,:,roi_index_human]

In [8]:
df_chimp = pd.concat([
    build_df_atlas(KL_human_chimp_LH, human_ids, chimp_ids, "human-chimpanzee", "BNA", "LH", region),
    build_df_atlas(KL_human_chimp_RH, human_ids, chimp_ids, "human-chimpanzee", "BNA", "RH", region)
])
df_macaque = pd.concat([
    build_df_atlas(KL_human_macaque_LH, human_ids, macaque_ids, "human-macaque", "BNA", "LH", region),
    build_df_atlas(KL_human_macaque_RH, human_ids, macaque_ids, "human-macaque", "BNA", "RH", region)
])
df_marmoset = pd.concat([
    build_df_atlas(KL_human_marmoset_LH, human_ids, marmoset_ids, "human-marmoset", "BNA", "LH", region),
    build_df_atlas(KL_human_marmoset_RH, human_ids, marmoset_ids, "human-marmoset", "BNA", "RH", region)
])
df_marmoset_vPaxinos = pd.concat([
    build_df_atlas(KL_human_marmoset_vPaxinos_LH, human_ids, marmoset_ids, "human-marmoset", "Paxinos", "LH", region),
    build_df_atlas(KL_human_marmoset_vPaxinos_RH, human_ids, marmoset_ids, "human-marmoset", "Paxinos", "RH", region)
])

df = pd.concat([df_chimp, df_macaque, df_marmoset_vPaxinos], ignore_index=True)
print(df.head())
print(df.shape)  # 应该是 (40*46*2 + 40*8*2 + 40*24*2) = 6240 条记录

df.to_csv(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/af_projection/result/minKL_indi_{region}_vPaxinos.csv', index=False)

df = pd.concat([df_marmoset, df_marmoset_vPaxinos], ignore_index=True)
print(df.head())
print(df.shape)  # 应该是 (40*46*2 + 40*8*2 + 40*24*2) = 6240 条记录

df.to_csv(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/af_projection/result/minKL_indi_{region}_vPaxinos_marmoset.csv', index=False)

  human_id nonhuman_id           species atlas hemisphere region        KL
0   100307      Agatha  human-chimpanzee   BNA         LH   A45c  1.539080
1   100307      Amanda  human-chimpanzee   BNA         LH   A45c  2.418905
2   100307     Artemus  human-chimpanzee   BNA         LH   A45c  3.334183
3   100307      Azalea  human-chimpanzee   BNA         LH   A45c  1.959067
4   100307     Barbara  human-chimpanzee   BNA         LH   A45c  1.287634
(6240, 7)
  human_id nonhuman_id         species atlas hemisphere region        KL
0   100307  sub-NIHm14  human-marmoset   BNA         LH   A45c  0.671537
1   100307  sub-NIHm15  human-marmoset   BNA         LH   A45c  0.660150
2   100307  sub-NIHm16  human-marmoset   BNA         LH   A45c  0.770637
3   100307  sub-NIHm17  human-marmoset   BNA         LH   A45c  0.730925
4   100307  sub-NIHm19  human-marmoset   BNA         LH   A45c  0.581099
(3840, 7)


# construct MixedLM model

In [9]:
# # ==========================================================
# # 1) 假设你已经有一个长表 df
# #    包含: human_id, nonhuman_id, species, hemisphere, KL
# # ==========================================================

# # 确保类型
# df["species"] = df["species"].astype("category")
# df["hemisphere"] = df["hemisphere"].astype("category")
# df["human_id"] = df["human_id"].astype("category")
# df["nonhuman_id"] = df["nonhuman_id"].astype("category")

# df["species"] = df["species"].cat.reorder_categories(["human-marmoset", "human-chimpanzee", "human-macaque"])

In [10]:
# # ----------------------------------------------------------
# # 2) MixedLM 模型：KL ~ species * hemisphere
# #    随机效应: (1 | human_id) + (1 | nonhuman_id)
# # ----------------------------------------------------------
# vc = {"nonhuman": "0 + C(nonhuman_id)"}  # 非人个体随机截距

# model = smf.mixedlm(
#     "KL ~ species * hemisphere",
#     data=df,
#     groups=df["human_id"],   # (1 | human_id)
#     vc_formula=vc
# )
# res = model.fit(method="lbfgs", reml=True)
# print(res.summary())

# Wald comparison

In [11]:
# # ----------------------------------------------------------
# # 3) Wald 对比函数
# #    用于比较任意两个物种-半球组合
# # ----------------------------------------------------------
# colnames = res.model.exog_names
# print("Fixed-effect columns:", colnames)

# def mean_vector(species, hemisphere):
#     """
#     返回 species × hemisphere 的固定效应均值在设计矩阵上的表达
#     基线: marmoset-LH
#     """
#     v = np.zeros(len(colnames))
#     v[colnames.index("Intercept")] = 1.0

#     # species 效应
#     if species == "human-chimpanzee":
#         v[colnames.index("species[T.human-chimpanzee]")] = 1.0
#     elif species == "human-macaque":
#         v[colnames.index("species[T.human-macaque]")] = 1.0

#     # hemisphere 效应
#     if hemisphere == "RH":
#         v[colnames.index("hemisphere[T.RH]")] = 1.0
#         # 交互效应
#         if species == "human-chimpanzee":
#             v[colnames.index("species[T.human-chimpanzee]:hemisphere[T.RH]")] = 1.0
#         elif species == "human-macaque":
#             v[colnames.index("species[T.human-macaque]:hemisphere[T.RH]")] = 1.0
#     return v


# def wald_between_species(species_a, hemi_a, species_b, hemi_b):

#     """比较两个组合的差异 (species_a-hemi_a) - (species_b-hemi_b)"""

#     c = mean_vector(species_a, hemi_a) - mean_vector(species_b, hemi_b)
#     c = c.reshape(1, -1)   # 必须二维

#     test = res.t_test(c)
#     return {
#         "contrast": f"{species_a}-{hemi_a} vs {species_b}-{hemi_b}",
#         "diff": float(test.effect),
#         "se": float(test.sd),
#         "t": float(test.tvalue),
#         "p": float(test.pvalue),
#         "ci_low": float(test.conf_int()[0][0]),
#         "ci_high": float(test.conf_int()[0][1]),
#     }


# def wald_between_hemi(species):

#     """比较物种内的左右差异 (LH - RH)"""

#     v_LH = mean_vector(species, "LH")
#     v_RH = mean_vector(species, "RH")
#     c = (v_LH - v_RH).reshape(1, -1)

#     test = res.t_test(c)
#     return {
#         "contrast": f"{species}: LH - RH",
#         "diff": float(test.effect),
#         "se": float(test.sd),
#         "t": float(test.tvalue),
#         "p": float(test.pvalue),
#         "ci_low": float(test.conf_int()[0][0]),
#         "ci_high": float(test.conf_int()[0][1]),
#     }


# def wald_interaction(species_a, species_b):

#     """交互作用（左右差异在物种之间的差）：(LH-RH)_species_a - (LH-RH)_species_b""" 

#     c = (mean_vector(species_a,"LH") - mean_vector(species_a,"RH")
#          - mean_vector(species_b,"LH") + mean_vector(species_b,"RH")).reshape(1, -1)
#     test = res.t_test(c)
#     return {
#         "contrast": f"(LH-RH)_{species_a} - (LH-RH)_{species_b}",
#         "diff": float(test.effect),
#         "se": float(test.sd),
#         "t": float(test.tvalue),
#         "p": float(test.pvalue),
#         "ci_low": float(test.conf_int()[0][0]),
#         "ci_high": float(test.conf_int()[0][1]),
#     }

In [12]:
# contrasts_between_species = [
#     ("human-chimpanzee", "LH", "human-macaque", "LH"),
#     ("human-chimpanzee", "RH", "human-macaque", "RH"),
#     ("human-marmoset", "LH", "human-macaque", "LH"),
#     ("human-marmoset", "RH", "human-macaque", "RH"),
#     ("human-marmoset", "LH", "human-chimpanzee", "LH"),
#     ("human-marmoset", "RH", "human-chimpanzee", "RH"),
# ]

# wald_results_between_species = pd.DataFrame([wald_between_species(*c) for c in contrasts_between_species])
# rej, p_adj, _, _ = multipletests(wald_results_between_species["p"].values, alpha=0.05, method="fdr_bh")
# wald_results_between_species["p_adj_fdr"] = p_adj
# wald_results_between_species["sig_fdr"] = rej
# print("\nWald pairwise contrasts between species:")
# print(wald_results_between_species)

In [13]:
# contrasts_between_hemispheres = ["human-marmoset", "human-chimpanzee", "human-macaque"]

# wald_results_between_hemispheres = pd.DataFrame([wald_between_hemi(c) for c in contrasts_between_hemispheres])
# rej, p_adj, _, _ = multipletests(wald_results_between_hemispheres["p"].values, alpha=0.05, method="fdr_bh")
# wald_results_between_hemispheres["p_adj_fdr"] = p_adj
# wald_results_between_hemispheres["sig_fdr"] = rej
# print("\nWald pairwise contrasts between hemispheres (LH-RH):")
# print(wald_results_between_hemispheres)


In [14]:
# contrasts_interaction = [
#     ("human-chimpanzee", "human-marmoset"),
#     ("human-macaque", "human-marmoset"),
#     ("human-chimpanzee", "human-macaque")
# ]

# wald_results_interaction = pd.DataFrame([wald_interaction(a,b) for a,b in contrasts_interaction])
# rej, p_adj, _, _ = multipletests(wald_results_interaction["p"].values, alpha=0.05, method="fdr_bh")
# wald_results_interaction["p_adj_fdr"] = p_adj
# wald_results_interaction["sig_fdr"] = rej
# print("\nWald pairwise contrasts between species * hemispheres (LH-RH) interaction:")
# print(wald_results_interaction)